<a href="https://colab.research.google.com/github/luconsultoria/Telecom-X/blob/main/TelecomX_BR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#📌 Extracão

In [ ]:
# Importação de bibliotecas
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Configuração de visualização
sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

# Supondo que raw_data deve ser carregado de um arquivo chamado 'data.json'.
# Substituindo 'data.json' pelo caminho real do arquivo se for diferente,
tentar:
    with open('data.json', 'r') as f:
        raw_data = json.load(f)
except FileNotFoundError:
    print("Error: 'data.json' not found. Please provide the correct file path for the raw data.")
    raw_data = [] # Inicialize com uma lista vazia para evitar erros futuros

df = pd.json_normalize(raw_data)

Error: 'data.json' not found. Please provide the correct file path for the raw data.


## EXTRAÇÃO E CONFIGURAÇÃO DOS DADOS ##
Este notebook apresenta uma análise de churn (evasão de clientes) da empresa TelecomX, através da extração e configuração dos dados de uma API, com o objetivo de identificar padrões nos dados de clientes que cancelaram os serviços e fornecer recomendações para ajudar na retenção de usuários.


#🔧 Transformação

In [ ]:
# Renomear colunas para remover prefixos
df.columns = [col.replace("customer.", "").replace("phone.", "").replace("internet.", "").replace("account.", "").replace("Charges.", "Charges_") for col in df.columns]

# Padronizar categorias
cols_padronizar = ['OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','MultipleLines']
for col in cols_padronizar:
    if col in df.columns:
        df[col] = df[col].replace({'No internet service': 'No', 'No phone service': 'No'})

# Converter variáveis binárias
binarias = ['SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']
for col in binarias:
    if col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].map({'Yes': 1, 'No': 0})

df_final = df.copy()

## Transformação dos Dados
Nesta etapa, foi realizado a limpeza, normalização e conversão de tipos para garantir uma base de dados confiável para a análise.


#📊 Carga e análise

In [ ]:
df_final = df.copy()

# Converter a coluna 'Churn' para numérica (0 para 'Não' e 1 para 'Sim')
df_final['Churn'] = df_final['Churn'].map({'Yes': 1, 'No': 0})

# Taxa de churn
taxa_churn = df_final['Churn'].mean() * 100
print("Taxa de churn (%):", round(taxa_churn, 2))

# Distribuição de churn
sns.countplot(data=df_final, x='Churn')
plt.title('Distribuição de Churn')
plt.xticks([0, 1], ['Não', 'Sim'])
plt.show()

# Contrato x Churn
sns.countplot(data=df_final, x='Contract', hue='Churn')
plt.title('Churn por Tipo de Contrato')
plt.show()

# Forma de pagamento x Churn
sns.countplot(data=df_final, x='PaymentMethod', hue='Churn')
plt.xticks(rotation=45)
plt.title('Churn por Forma de Pagamento')
plt.show()

# Análise de Tenure
print("\nEstatísticas de Tenure por categoria de Churn:")
print(df_final.groupby('Churn')['tenure'].describe())
print("\nValores únicos de Tenure por categoria de Churn:")
print(df_final.groupby('Churn')['tenure'].nunique())
print("\nValores ausentes em Tenure por categoria de Churn:")
print(df_final.groupby('Churn')['tenure'].apply(lambda x: x.isnull().sum()))

# Distribuição de Tenure
sns.histplot(data=df_final, x='tenure', hue='Churn', bins=30, kde=True)
plt.title('Distribuição de Tenure por Churn')
plt.show()

# Tipo de internet x Churn
sns.countplot(data=df_final, x='InternetService', hue='Churn')
plt.title('Churn por Tipo de Internet')
plt.show()

# Gastos Totais x Churn
sns.boxplot(data=df_final, x='Churn', y='Charges_Total')
plt.title('Gastos Totais por Churn')
plt.xticks([0, 1], ['Não', 'Sim'])
plt.show()

# Correlação com Churn
corr = df_final.corr(numeric_only=True)
sns.heatmap(corr[['Churn']].sort_values(by='Churn', ascending=False), annot=True, cmap='coolwarm')
plt.title('Correlação com Churn')
plt.show()

## ANALISAR E INVESTIGAR O CHURN ##
Investigar a distribuição do churn e sua relação com outras variáveis.

# * Distribuição do Churn #
•	Cerca de 26% dos clientes abandonaram a empresa.

•	Alta taxa para um serviço de assinatura.

# * Perfil dos Clientes com Maior Evasão #
•	Clientes com contratos Month-to-month têm a maior taxa de churn.

•	Electronic check como forma de pagamento está fortemente associada a maior evasão.

•	Clientes com tenure (tempo de contrato) menor que 12 meses têm maior propensão ao churn.

•	Usuários de internet via Fiber optic também apresentam maior evasão.


#📄Relatorio Final

## Principais Achados( INSIGHTS): ##

* Contratos mensais são vulneráveis à evasão.

* Clientes com baixa fidelização (baixo tenure) são mais voláteis.

* Alto valor de cobrança mensal não necessariamente aumenta o churn, mas combinado com contratos curtos e cobrança eletrônica, sim.

* A falta de serviços adicionais (como suporte técnico ou segurança online) também influencia negativamente.

## Sugestões para Reduzir o Churn (CONCLUSÕES) ##

* Criar campanhas de fidelização para clientes com menos de 12 meses de contrato.

* Incentivar planos de contrato anual ou bienal com descontos.

* Oferecer benefícios ou pacotes promocionais para quem usa pagamento por Electronic check.

* Estimular o uso de serviços adicionais como segurança online e suporte técnico.

* Implementar um sistema de alerta precoce para clientes com alta propensão de evasão (com base nos padrões identificados).

